In [12]:
import sys, os
parent_dir = os.getcwd()
path = os.path.dirname(parent_dir)
sys.path.append(path)

from gym_homer.envs.test_env_v0 import HomerEnv
import pandas as pd
import numpy as np

import gym, torch, numpy as np, torch.nn as nn
from tianshou.utils import WandbLogger
from torch.utils.tensorboard import SummaryWriter
import tianshou as ts
from gym import spaces, wrappers

In [13]:
# Load Data
data = pd.read_csv(path+"/test_env_data.csv", index_col=False).fillna(0)
pd.DataFrame(data)

,time_x,time_y,weekend,month_x,month_y,region_1,region_2,region_3,solar,loads,import_tariff,export_tariff,max_d,max_c,soc
0,0.000,1.000,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
1,0.500,0.866,0,0.0,1.0,0,0,0,-1,0.5,0.05,0.0,0.0,0.0,0.0
2,0.866,0.500,0,0.0,1.0,0,0,0,-1,0.5,0.05,0.0,0.0,0.0,0.0
3,1.000,0.000,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
4,0.866,-0.500,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
5,0.500,-0.866,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
6,0.000,-1.000,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
7,-0.500,-0.866,0,0.0,1.0,0,0,0,0,0.5,2.00,2.0,0.0,0.0,0.0
8,-0.866,-0.500,0,0.0,1.0,0,0,0,0,0.5,2.00,2.0,0.0,0.0,0.0
9,-1.000,-0.000,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0


In [22]:
#task = 'HomerEnv-v0'
env = HomerEnv(data=data, start_soc='random')
lr, epoch, batch_size = 1e-3, 20, 64
train_num, test_num = 10, 100
gamma, n_step, target_freq = 0.9, 3, 320
buffer_size = 20000
eps_train, eps_test = 0.1, 0.05
step_per_epoch, step_per_collect = 10000, len(data) * 10
logger = ts.utils.TensorboardLogger(SummaryWriter('log/dqn'))  # TensorBoard is supported!
# For other loggers: https://tianshou.readthedocs.io/en/master/tutorials/logger.html

# you can also try with SubprocVectorEnv
train_envs = ts.env.DummyVectorEnv([lambda: HomerEnv(data=data) for _ in range(train_num)])
test_envs = ts.env.DummyVectorEnv([lambda: HomerEnv(data=data) for _ in range(test_num)])

from tianshou.utils.net.common import Net
# you can define other net by following the API:
# https://tianshou.readthedocs.io/en/master/tutorials/dqn.html#build-the-network
#env = gym.make('gym_homer/HomerEnv-v0', data=data)
wrapped_env = wrappers.FlattenObservation(env)
state_shape = wrapped_env.observation_space.shape or wrapped_env.observation_space.n
action_shape = wrapped_env.action_space.shape or wrapped_env.action_space.n
net = Net(state_shape=state_shape, action_shape=action_shape, hidden_sizes=[64, 128, 8])
optim = torch.optim.Adam(net.parameters(), lr=lr)

policy = ts.policy.DQNPolicy(net, optim, gamma, n_step, target_update_freq=target_freq)
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(buffer_size, train_num), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)  # because DQN uses epsilon-greedy method

In [23]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector, epoch, 
    step_per_epoch, step_per_collect,
    test_num, batch_size, update_per_step=1 / step_per_collect,
    train_fn=lambda epoch, env_step: policy.set_eps(eps_train),
    test_fn=lambda epoch, env_step: policy.set_eps(eps_test),
    stop_fn=lambda mean_rewards: mean_rewards >= 2,
    logger=logger)
print(f'Finished training! Use {result["duration"]}')

Epoch #1: 10080it [00:00, 13032.56it/s, env_step=10080, len=11, loss=0.435, n/ep=10, n/st=120, rew=1.34]                         


Epoch #1: test_reward: 1.605500 ± 1.201544, best_reward: 1.605500 ± 1.201544 in #1


Epoch #2: 10080it [00:00, 13144.20it/s, env_step=20160, len=11, loss=0.227, n/ep=10, n/st=120, rew=1.48]                          


Epoch #2: test_reward: 1.717250 ± 0.804369, best_reward: 1.717250 ± 0.804369 in #2


Epoch #3: 10080it [00:00, 13345.23it/s, env_step=30240, len=11, loss=0.218, n/ep=10, n/st=120, rew=0.43]                          


...

Epoch #18: test_reward: 1.763000 ± 0.756038, best_reward: 1.879750 ± 0.442376 in #17


Epoch #19: 10080it [00:00, 13244.88it/s, env_step=191520, len=11, loss=0.202, n/ep=10, n/st=120, rew=1.16]                           


Epoch #19: test_reward: 1.662750 ± 0.995085, best_reward: 1.879750 ± 0.442376 in #17


Epoch #20: 10080it [00:00, 13551.99it/s, env_step=201600, len=11, loss=0.176, n/ep=10, n/st=120, rew=1.76]                          


Epoch #20: test_reward: 1.784000 ± 0.804499, best_reward: 1.879750 ± 0.442376 in #17
Finished training! Use 16.26s


In [24]:
policy.eval()
policy.set_eps(20)
collector = ts.data.Collector(policy, env)
collector.collect(n_episode=10)

{'n/ep': 10,
 'n/st': 110,
 'rews': array([ 1.875     ,  1.95      ,  1.875     , -2.12529535,  1.19425189,
         1.95      ,  1.875     , -1.7076813 ,  1.925     ,  1.975     ]),
 'lens': array([11, 11, 11, 11, 11, 11, 11, 11, 11, 11]),
 'idxs': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'rew': 1.0786275231075706,
 'len': 11.0,
 'rew_std': 1.5160159952057055,
 'len_std': 0.0}